# Preprocessing

In [1]:
import os
import bestreads.text as text
import pandas as pd

data = pd.read_csv('./data/goodreads_books.csv')
data_train = data.sample(frac = 0.9, random_state = 111)
data_test = data.drop(data_train.index)

## Train-Test Set Splits

In [2]:
save_dir = './data/processed/'
os.makedirs(save_dir, exist_ok=True)
data_train.to_csv(save_dir + 'goodreads_books_train.csv', index = False)
data_test.to_csv(save_dir + 'goodreads_books_test.csv', index = False)
data_train.reset_index(inplace=True)
data_test.reset_index(inplace=True)

## Text Cleaning

In [3]:
is_english_description_train = text.is_english(data_train['description'])
english_descriptions_train = data_train.description[is_english_description_train]
cleaned_descriptions_train = text.clean_text(english_descriptions_train)
is_english_description_train = is_english_description_train.rename('is_english_description',)
cleaned_descriptions_train = cleaned_descriptions_train.rename('cleaned_descriptions')

is_english_description_test = text.is_english(data_test['description'])
english_descriptions_test = data_test.description[is_english_description_test]
cleaned_descriptions_test = text.clean_text(english_descriptions_test)
is_english_description_test = is_english_description_test.rename('is_english_description')
cleaned_descriptions_test = cleaned_descriptions_test.rename('cleaned_descriptions')

## Grouping Descriptions by Genre

In [4]:
genre_and_votes_train = text.get_genres(data_train.genre_and_votes[is_english_description_train])
genre_and_votes_test = text.get_genres(data_test.genre_and_votes[is_english_description_test])
combined = text.combine_genres(genre_and_votes_train.genre_1, cleaned_descriptions_train)

/home/adam/Documents/bestreads/bestreads/text.py:142: RuntimeWarning: NaN values detected in genre_and_votes; these will beskipped
  + 'skipped', category=RuntimeWarning)
100%|██████████████████████████████████████████████████| 4409/4409 [00:00<00:00, 15330.36it/s]


## Calculating TF-IDF

In [5]:
tf_idf_table_train = text.tf_idf(combined)

# Here, we save the index because each index is a word
tf_idf_table_train.to_csv(save_dir + 'tf_idf_table_train.csv', index_label='word')
tf_idf_table_train.head()

100%|█████████████████████████████████████████████████████████| 60/60 [00:14<00:00,  4.13it/s]


,Religion,Christian,Westerns,Historical,Animals,Spirituality,Psychology,Science Fiction,Plays,Realistic Fiction,...,Adult Fiction,Travel,Philosophy,Poetry,History,Adventure,Paranormal,Thriller,Mystery,Sequential Art
much,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
public,0.000036,0.000018,0.000015,0.000013,0.000012,0.000014,0.000070,0.000010,0.000037,0.000051,...,0.000033,0.000025,0.000074,0.000033,0.000042,0.000014,0.000000,0.000014,0.000012,0.000017
debat,0.000323,0.000000,0.000000,0.000015,0.000000,0.000000,0.000258,0.000012,0.000000,0.000000,...,0.000000,0.000062,0.000566,0.000074,0.000199,0.000000,0.000000,0.000000,0.000007,0.000006
unit,0.000084,0.000029,0.000000,0.000105,0.000056,0.000000,0.000100,0.000148,0.000070,0.000000,...,0.000000,0.000194,0.000027,0.000063,0.000365,0.000100,0.000016,0.000233,0.000064,0.000069
state,0.000098,0.000025,0.000000,0.000067,0.000033,0.000019,0.000206,0.000089,0.000103,0.000000,...,0.000000,0.000142,0.000145,0.000064,0.000316,0.000118,0.000009,0.000151,0.000056,0.000047


In [6]:
# Save the training data
processed_data_train = (is_english_description_train.to_frame()
                        .merge(cleaned_descriptions_train,
                               left_index=True, right_index=True,
                               validate='1:1')
                        .merge(genre_and_votes_train,
                               left_index=True, right_index=True,
                               validate='1:1'))

# Here, we save the index because some rows are now missing due to
# english language selection
processed_data_train.to_csv(save_dir + 'goodreads_books_train_processed.csv', 
                            index_label='index')

# Save the test data
processed_data_test = (is_english_description_test.to_frame()
                        .merge(cleaned_descriptions_test,
                               left_index=True, right_index=True,
                               validate='1:1')
                        .merge(genre_and_votes_test,
                               left_index=True, right_index=True,
                               validate='1:1'))

# Here, we save the index because some rows are now missing due to
# english language selection
processed_data_test.to_csv(save_dir + 'goodreads_books_test_processed.csv', 
                           index_label='index')